In [1]:
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
import json

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [2]:
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

In [3]:
session = get_session('EMPLOYEE',"fd0cf79c-8118-43a7-8f0b-059e8f78227a")

In [5]:
from snowflake.ml.registry.registry import Registry

In [ ]:

  "configuration": {"filter_condition": "",
  "frequency": {"frequencyType":"ONCE"},
  "input_dataset": "EMPLOYEE_TEST",
  "write_strategy": "New Table"
},
  "deployment_type": "BATCH",
  "model_name": "MODEL_5AABC4FE_FC3D_4121_9E11_52C83C9FECC9_FDC_PRAKHAR_TEST",
  "version": "V_110405"
}

In [14]:
reg = Registry(session=session)
m = reg.get_model('MODEL_5AABC4FE_FC3D_4121_9E11_52C83C9FECC9_FDC_PRAKHAR_TEST')
mv = m.version('V_110405')
# fetch_data_sql = "select * from " + input_table + " " + filter_cond
df = session.sql('select * from EMPLOYEE_TEST').to_pandas() 
print(df)
# print(type(df))
remote_prediction = mv.run(df, function_name="predict")
new_table_name = "prediction_" + 'output_table_batch'
session.write_pandas(remote_prediction, new_table_name, auto_create_table=True, overwrite=True)

      JOININGYEAR       AGE  PAYMENTTIER  EXPERIENCEINCURRENTDOMAIN  \
0        0.833333  0.631579          1.0                   0.000000   
1        0.166667  0.315789          0.0                   0.428571   
2        0.333333  0.842105          1.0                   0.285714   
3        0.666667  0.263158          1.0                   0.714286   
4        0.833333  0.105263          1.0                   0.285714   
...           ...       ...          ...                        ...   
3701     0.500000  0.526316          1.0                   0.142857   
3702     0.833333  0.473684          0.5                   0.285714   
3703     0.166667  0.157895          1.0                   0.428571   
3704     0.666667  0.421053          1.0                   0.285714   
3705     1.000000  0.263158          1.0                   0.714286   

      EDUCATION_0.0  EDUCATION_1.0  EDUCATION_2.0  CITY_0.0  CITY_1.0  \
0               1.0            0.0            0.0       1.0       0.0   
1

ValueError: (2112) Data Validation Error: feature "EDUCATION_0.0" does not exist in data.